<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
#!pip install imbalanced-learn -q
#!pip install swifter - q

#!pip install lightgbm
#!pip install xgboost

In [2]:
import pandas as pd
from tqdm import notebook
import re
from time import time 
#import nltk
import numpy as np
from numpy.random import default_rng
import transformers
from sklearn.model_selection import train_test_split,cross_val_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

from __future__ import unicode_literals, print_function
from spacy.lang.en import English

In [3]:
from imblearn.under_sampling import RandomUnderSampler

In [4]:
import torch
import transformers

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
from xgboost import XGBClassifier

In [7]:
import swifter

In [8]:
#import spacy
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#from spacy.pipeline import Lemmatizer
#!python -m spacy download en_core_web_sm
#nlp = spacy.load("en_core_web_sm")

In [9]:
#tokenizer = transformers.BertTokenizer(
#    vocab_file='datasets/ds_bert/vocab.txt')

In [10]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
nlp = English()
nlp.add_pipe('sentencizer')

In [12]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [13]:
tokenizer.encode('He ran. He fell.')

[101, 2002, 2743, 1012, 2002, 3062, 1012, 102]

In [14]:
tokenizer.added_tokens_encoder

{'[PAD]': 0, '[UNK]': 100, '[CLS]': 101, '[SEP]': 102, '[MASK]': 103}

In [15]:
for enc in tokenizer.encode('He ran. He fell. Aaaaaaaaaaa...'):
    display((enc,tokenizer.decode([f'{enc}'])))

(101, '[CLS]')

(2002, 'he')

(2743, 'ran')

(1012, '.')

(2002, 'he')

...

(11057, '##aa')

(1012, '.')

(1012, '.')

(1012, '.')

(102, '[SEP]')

In [16]:
RANDOM_STATE = 88

## Подготовка

In [17]:
start = time()
#df['text'].apply(lambda x: len(str(x).split()))
end = time()
end-start

0.0

In [18]:
url = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

In [19]:
df = pd.read_csv(url)

In [20]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [22]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [23]:
df['text'].duplicated().sum()

0

In [24]:
def clean_text(string):
    string = re.sub(r'[^a-z]', ' ', string, flags=re.IGNORECASE)
    string = string.split()
    string = ' '.join(string)
    return string.lower()

In [25]:
df['clean_text'] = df['text'].apply(lambda x:clean_text(x))

In [26]:
df['clean_text'].duplicated().sum()

1323

In [27]:
df.drop_duplicates(subset='clean_text', inplace=True)

In [28]:
df.reset_index(drop=True, inplace=True)

In [29]:
df['text'].str.lower().duplicated().sum()

0

In [30]:
df.isna().sum()

text          0
toxic         0
clean_text    0
dtype: int64

In [31]:
def sents(string):
    doc = nlp(string)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

In [32]:
df['word_count'] = df['text'].apply(lambda x: x.count(' ') + 1)

In [33]:
#df['sent_count'] = df['text'].apply(lambda x: len(sents(x)))

In [34]:
df['len'] = df['text'].apply(lambda x: len(x))

In [35]:
df['text'][154485]

'Turret #4\n\nWhy did the builder raise turret #4 on a high mount, while #3 sits on a normal low mount? It can only fire broadside, so why bother with added top weight? East of Borschov'

In [36]:
#df['text'][112719]

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157969 entries, 0 to 157968
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        157969 non-null  object
 1   toxic       157969 non-null  int64 
 2   clean_text  157969 non-null  object
 3   word_count  157969 non-null  int64 
 4   len         157969 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 6.0+ MB


In [38]:
#df['text_lower'] = df['mid_text'].str.lower()
#df.drop_duplicates(subset='text_lower', inplace=True)
#df.reset_index(drop=True, inplace=True)

In [39]:
def distribution_plt(df_list, name, val_list, plt_xlim=1):

    x_max = min([series.max() for series in df_list])* plt_xlim

    fig, ax = plt.subplots(2,1,figsize=(10,8))
    ax[0].boxplot([df for df in df_list], vert=False)

    ax[0].set_yticklabels(val_list)
    ax[0].set_xlabel(name)
    ax[0].set_xlim([0, x_max])

    ax[1].hist([df for df in df_list], histtype='stepfilled', bins=500)
    ax[1].set_ylabel('частота')
    ax[1].set_xlabel(name)
    ax[1].set_xlim([0, x_max])

    
    plt.suptitle(f'Ящики с усами и гистограмма для признака "{name}"')
    plt.legend(val_list);
    #plt.xlim((0,x_max))

In [40]:
name_dict = {'word_count' : 'количество слов',
 'sent_count' : 'количество предложений',
 'len' : 'количество символов'}

for col in ['word_count', 'sent_count', 'len']:
    
    distribution_plt(
        
        [df[df['toxic']==1][col], df[df['toxic']==0][col]],
        
        name_dict[col],
        
        ['токсичные\nкомментарии', 'не токсичные\nкомментарии'],
        
        0.1)

In [41]:
def word_rebuild(word_list):
    
    for word in range(len(word_list)):
        if len(word_list[word])>100:
            word_list[word] = ''.join(list(dict.fromkeys(word_list[word])))
    
    return word_list

In [42]:
config = transformers.BertConfig.from_dict({
      "attention_probs_dropout_prob": 0.1,
      "directionality": "bidi",
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.1,
      "hidden_size": 768,
      "initializer_range": 0.02,
      "intermediate_size": 3072,
      #"layer_norm_eps": 7e-2,
      "layer_norm_eps": 1e-12,
      #"layer_norm_eps": 4e-5,
      "max_position_embeddings": 512,
      "model_type": "bert",
      "num_attention_heads": 12,
      "num_hidden_layers": 12,
      "pad_token_id": 0,
      "pooler_fc_size": 768,
      "pooler_num_attention_heads": 12,
      "pooler_num_fc_layers": 3,
      "pooler_size_per_head": 128,
      "pooler_type": "first_token_transform",
      #"position_embedding_type": "absolute",
      #"transformers_version": "4.12.5",
      "use_cache":True,
      "type_vocab_size": 2,
      "vocab_size": 30522
}
)

def chunker(string, max_tokens):

    sents_ = sents(string)

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    
    if len(toks)>1:
        result = sum(toks,[])    
    else:
        result = toks[0]
    
    if len(result)>max_tokens:
        
        mid_sent = len(toks)//2
        
        if mid_sent == 1:
            
            if len(toks[1])<max_tokens:
                result = toks[1]    
            else:
                result = toks[0]
                
        elif mid_sent > 1:
            
            for s in range(1,mid_sent):

                mid_sents = toks[(mid_sent-s):mid_sent] + toks[mid_sent:(mid_sent+s)]#+toks[-1]
                mid_sents_s = sum(mid_sents,[])
                if len(mid_sents_s) > max_tokens:
                    break
                    result = sum(mid_sents[:-1],[])
        
        else:
            result = sum(toks,[])
    #return result
    return result[:max_tokens][:-1]+[102]

def chunker(string, max_tokens):

    max_tokens = max_tokens-1
    sents_ = sents(string)

    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=False) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    

    result = sum(toks,[])    
    
    if len(result)>max_tokens:
        if len(sents_)>2:
            result = toks[0]+toks[-1]
    
            
    return [101] + result[:max_tokens-2] + [102]

In [43]:
def chunker(string, max_tokens=512):
    sents_ = sents(string)
    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=False) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
        
    result = sum(toks, [])#[:-1]
    return result 

In [44]:
def chunker(string, vfile='bert-base-uncased', max_tokens=512):

    tokenizer = transformers.BertTokenizer.from_pretrained(vfile)
    
    tokenized = tokenizer.encode(string, add_special_tokens=True)

    result = tokenized[:max_tokens]

    return result

def chunker(string, max_tokens=512):

    sents_ = sents(string)

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    result = sum(toks, [])#[:-1]
    
    
    
    if len(set(result))>len(result)*0.7:
  
        result = list(dict.fromkeys(result))[:-1]
    
    
    result = result[:max_tokens-1]
    
    if result[-1]!=102:
        result = result[:max_tokens-1] + [102]

    
    
    return result

In [45]:
#df['text'][4704]

In [46]:
tokenizer.decode(chunker(df['text'][4704]))

"[CLS] dab said : my impression remainst that the only reason this article is so long and detailed is the attempt to disguise from the casual reader the fact that the entire thing can be shrugged off in a one - liner. yes, that's probably true, but what is the alternative? many of us have been battling to make the cmt clear, but every time a statement is made on how crazy the theory is, the other side asks for a proof / citation. and since that has happened almost every week for the last six months ( and probably longer ), the article has grown in size. i'm not sure how we can overcome this problem, but i'm open to suggestions. [SEP]"

In [47]:
#tokenizer.decode(tokenizer.encode(df['text'][4704]))

In [48]:
df[df['text'].str.contains('I ass. I ass.')]['text']

4700    do go fuck off bastard\nDo Yyou Have a life?\n...
Name: text, dtype: object

In [49]:
#tokenizer.decode(chunker(df['text'][4700], 512))

In [50]:
#df[df['text'].apply(lambda x: len(x))>=4900]['text'].apply(lambda x: len(chunker(x))).describe()

In [51]:
#tokenizer.decode(chunker(df['text'][82496]))

In [52]:
#df['text'].sample()

In [53]:
#df.sample(2)['text'].apply(lambda x: chunker(x,10)).iloc[0]

In [54]:
#df['text'][82496]

In [55]:
#df['text'][116571]

In [56]:
#

def chunker(string, max_tokens=512):


    
    tokenized = tokenizer.encode(string, add_special_tokens=False)
    len_wo_tokens = max_tokens-2
    
    if len(tokenized)> len_wo_tokens:
        
        result = tokenized[:(len_wo_tokens//2)] + tokenized[::-1][:(len_wo_tokens//2)][::-1]
    else:
        result = tokenized

    return [101] + result + [102]

In [57]:
#chunker(df['text'][0])

In [58]:
#OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
#Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [59]:
#max_batch = 512

In [60]:
#init_config = config

In [61]:
#batch_size = 512

tokenized = X_train.swifter.apply(lambda x: chunker(x,max_batch))
#tokenized = text.apply(lambda x: chunker(x,max_batch))

max_len = 0

for tok in tokenized.values:
    if len(tok) > max_len:
        max_len = len(tok)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)


model = transformers.BertModel.from_pretrained('bert-base-uncased', config=init_config)
#model.config.max_position_embeddings = max_batch


batch_size = batch_size

embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

train_features = np.concatenate(embeddings)

test_tokenized = X_test.swifter.apply(lambda x: chunker(x,max_batch))


max_len = 0
for tok in test_tokenized.values:
    if len(tok) > max_len:
        max_len = len(tok)

padded = np.array([i + [0]*(max_len - len(i)) for i in test_tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

test_embeddings = []


for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            test_batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        test_embeddings.append(test_batch_embeddings[0][:,0,:].numpy())

test_features = np.concatenate(test_embeddings)

In [62]:
#train_features.shape

In [63]:
#test_features.shape

In [64]:
class bert_emb(object):


    def __init__(self, data, init_config,  batch_size=32, max_batch=512):

        self.data = data
        self.batch_size = batch_size
        self.max_batch = max_batch
        self.init_config = init_config 

        #self.config.max_position_embeddings = max_batch
        self.model = transformers.BertModel.from_pretrained('bert-base-uncased', config=init_config)
    
    def embedder(self):

        tokenized = self.data.swifter.apply(lambda x: chunker(x, self.max_batch))
        #tokenized = self.data.apply(lambda x:list(tokenizer(x,add_special_tokens=True)))#.iloc[:self.max_batch]
        #tokenized = self.data.apply(lambda x: tokenizer(x,add_special_tokens=True))
        #tokenized = tokenized.values.tolist()
        
        max_len = 0
        
        for tok in tokenized.values:
            if len(tok) > max_len:
                max_len = len(tok)
    
        #padded = [i + [0]*(max_len - len(i)) for i in tokenized.values]
        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])#,dtype=np.int64)
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch_size = self.batch_size
        
        embeddings = []
        
        for i in notebook.tqdm(range(len(padded) // batch_size)):
                batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
                attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
                
                with torch.no_grad():
                    batch_embeddings = self.model(batch, attention_mask=attention_mask_batch)
                
                embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
        features = np.concatenate(embeddings)

        return features
        

In [65]:
#bert_emb(df['text'][0:5],config,1).embedder().shape

In [175]:
def logy(feats, target):
    lr = LogisticRegression(random_state=RANDOM_STATE,max_iter=feats.shape[1],class_weight='balanced')
    lr.fit(feats, target)
    scores = cross_val_score(lr, feats, target, cv=5, scoring='f1_macro')
    return scores

In [67]:
#bert_emb(df['text'][0:1],config,batch_size=1,max_batch=10).embedder()[0]==bert_emb(df['text'][0:5],config,batch_size=1,max_batch=10).embedder()[0]

In [68]:
1024*32

32768

In [69]:
1024*16

16384

In [70]:
1024*8

8192

In [71]:
#best - 32768

In [72]:
df.shape

(157969, 5)

In [168]:
samp = df.sample(n=1024, random_state=RANDOM_STATE)

In [169]:
X_train, X_test, y_train, y_test = train_test_split(
    samp['text'],
    samp['toxic'],
    test_size = 0.50,
    random_state = RANDOM_STATE,
    stratify = samp['toxic'])

In [170]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((512,), (512,), (512,), (512,))

In [76]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train))

In [77]:
X_resample.shape, y_resample.shape

((88, 1), (88, 1))

In [78]:
train_sample = pd.DataFrame(X_resample).join(pd.DataFrame(y_resample)).sample(n=X_resample.shape[0], random_state=RANDOM_STATE)

In [79]:
X_resample = train_sample['text']
y_resample = train_sample['toxic']

In [80]:
X_resample.head(2), y_resample.head(2)

(68104     .\n\n i have just changed to Turkic again. i d...
 151100                              Fuc You! \n\nFY!\nS.HIT
 Name: text, dtype: object,
 68104     1
 151100    1
 Name: toxic, dtype: int64)

In [81]:
X_resample.tail(2), y_resample.tail(2)

(15603    Some asshole keeps changing the dates\n\nAdvan...
 6764                          Another 4-6 weeks then gone.
 Name: text, dtype: object,
 15603    1
 6764     0
 Name: toxic, dtype: int64)

In [82]:
config = transformers.BertConfig.from_dict({
      "attention_probs_dropout_prob": 0.1,
      "directionality": "bidi",
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.1,
      "hidden_size": 768,
      "initializer_range": 0.02,
      "intermediate_size": 3072,
      "layer_norm_eps": 1e-12,
      "max_position_embeddings": 512,
      "model_type": "bert",
      "num_attention_heads": 12,
      "num_hidden_layers": 12,
      "pad_token_id": 0,
      "pooler_fc_size": 768,
      "pooler_num_attention_heads": 12,
      "pooler_num_fc_layers": 3,
      "pooler_size_per_head": 128,
      "pooler_type": "first_token_transform",
      "type_vocab_size": 2,
      "vocab_size": 30522
}
)

X_resample = X_resample['text'].sort_index()
y_resample = y_resample['toxic'].sort_index()

In [83]:
X_resample.head(2), X_resample.tail(2)

(68104     .\n\n i have just changed to Turkic again. i d...
 151100                              Fuc You! \n\nFY!\nS.HIT
 Name: text, dtype: object,
 15603    Some asshole keeps changing the dates\n\nAdvan...
 6764                          Another 4-6 weeks then gone.
 Name: text, dtype: object)

In [84]:
y_resample.head(), y_resample.tail()

(68104     1
 151100    1
 108253    0
 33524     1
 100864    1
 Name: toxic, dtype: int64,
 74751     1
 40182     0
 147241    1
 15603     1
 6764      0
 Name: toxic, dtype: int64)

In [85]:
X_resample.shape

(88,)

In [86]:
778/2

389.0

In [87]:
def chunker(string, max_tokens=512):
    
    sents_ = sents(string)
    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]
    
    usents=[]
    
    for sent in tokenized_sents:
        if sent not in usents:
            usents.append(sent)
        
    result = sum(usents, [])
    
    
    return result 
    #return tokenized_sents

In [88]:
df['text'][0:2].apply(lambda x: chunker(x))

0    [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1    [101, 1040, 1005, 22091, 2860, 999, 102, 101, ...
Name: text, dtype: object

In [89]:
df['text'][0]#.sample(2).apply(lambda x: chunker(x))

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [90]:
class BertTokenizer(object):

    def __init__(self, text=[]):
        self.text = text

        # For DistilBERT:
        self.model_class, self.tokenizer_class, self.pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

        # Load pretrained model/tokenizer
        self.tokenizer = self.tokenizer_class.from_pretrained(self.pretrained_weights)

        self.model = self.model_class.from_pretrained(self.pretrained_weights)

    def get(self):

        df = pd.DataFrame(data={"text":self.text})
        tokenized = df["text"].swifter.apply((lambda x: self.tokenizer.encode(x, add_special_tokens=True)[:512]))

        max_len = 0
        for i in tokenized.values:
            if len(i) > max_len:
                max_len = len(i)

        padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded)
        attention_mask = torch.tensor(attention_mask)

        with torch.no_grad(): last_hidden_states = self.model(input_ids, attention_mask=attention_mask)
        
        features = last_hidden_states[0][:, 0, :].numpy()

        return features

In [172]:
def chunker(string):
    
    sents_ = sents(string)
    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_] #####

    usents=[]
    
    for sent in tokenized_sents:
        if sent not in usents:
            usents.append(sent)
    if len(usents)>1:
       result =  sum(usents, [])
    elif len(usents)==1:
        result = usents
    else:
        result = tokenized_sents[0]
        
    
    
    return [101] + result[1:-1][:510] + [102]

In [182]:
def chunker(string):
    
    sents_ = sents(string)
    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=False) for sent in sents_] #####

    usents=[]
    
    for sent in tokenized_sents:
        if sent not in usents:
            usents.append(sent)
    if len(usents)>1:
       result =  sum(usents, [])
    elif len(usents)==1:
        result = usents[0]
    else:
        result = tokenized_sents[0]
        
    
    
    return [101] + result[:510] + [102]

In [183]:
chunker(df['text'][0])

[101,
 7526,
 2339,
 1996,
 10086,
 2015,
 2081,
 2104,
 2026,
 5310,
 18442,
 13076,
 12392,
 2050,
 5470,
 2020,
 16407,
 1029,
 2027,
 4694,
 1005,
 1056,
 3158,
 9305,
 22556,
 1010,
 2074,
 8503,
 2006,
 2070,
 3806,
 2044,
 1045,
 5444,
 2012,
 2047,
 2259,
 14421,
 6904,
 2278,
 1012,
 1998,
 3531,
 2123,
 1005,
 1056,
 6366,
 1996,
 23561,
 2013,
 1996,
 2831,
 3931,
 2144,
 1045,
 1005,
 1049,
 3394,
 2085,
 1012,
 6486,
 1012,
 16327,
 1012,
 4229,
 1012,
 2676,
 102]

In [157]:
class BertTokenizer(object):

    def __init__(
        self,
        #tokenizer,
        #text=[],
        text,
        batch_size=1):
        
        self.text = text
        self.batch_size = batch_size
        self.model_class, self.pretrained_weights = (
            transformers.BertModel, 'bert-base-uncased')

        self.model = self.model_class.from_pretrained(self.pretrained_weights)

    
    def get(self):

        #df = pd.DataFrame(data={"text":self.text})
        #tokenized = self.text.swifter.apply((lambda x: self.tokenizer(x,self.pretrained_weights)[:512]))
        #tokenized = self.text.apply(lambda x: self.tokenizer(x,self.pretrained_weights)[:512])
        #tokenized = df["text"].apply(lambda x: self.tokenizer(x,self.pretrained_weights)[:512])
        #tokenized = self.text.apply(lambda x: self.tokenizer(x, add_special_tokens=1)[:512])
        tokenized = self.text.apply(lambda x: chunker(x))
        max_len = 0
        
        for tok in tokenized.values:
            if len(tok) > max_len:
                max_len = len(tok)

        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch_size = self.batch_size
        
        embeddings = []
        
        for i in notebook.tqdm(range(len(padded) // batch_size)):
                batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
                attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
                
                with torch.no_grad():
                    batch_embeddings = self.model(batch, attention_mask=attention_mask_batch)
                
                embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
        features = np.concatenate(embeddings)

        return features

In [74]:
X_resample.shape[0]/16

5.5

In [136]:
df['text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [ ]:
0.0020268327

In [184]:
BertTokenizer(X_train.iloc[0:10],5).get()[0][0]

  0%|          | 0/2 [00:00<?, ?it/s]

-0.17412826

In [132]:
len(chunker(X_train.iloc[0]))

76

In [75]:
98*16

1568

In [76]:
sents(df['text'][0])

['Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted?',
 "They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC.",
 "And please don't remove the template from the talk page since I'm retired now.89.205.38.27"]

In [77]:
from scipy.spatial.distance import cosine
#d = cosine(a, b)

cosine(BertTokenizer(
    text=df['text'][0:2],batch_size=1).get()[0],BertTokenizer(
    text=df['text'][0:2],batch_size=1).get()[1])

In [78]:
from scipy.spatial.distance import euclidean

def chunker(string, max_tokens=512):
    
    sents_ = sents(string)

    if len(sents_) > 1:
        
        tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]

        max_len = 0
        
        for token in tokenized_sents:
            if len(token) > max_len:
                max_len = len(token)
        
    
        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized_sents])
        #attention_mask = np.where(padded != 0, 1, 0)
    
        lst = []
        
        for i in range(len(padded)):
            if i!=len(padded)-1:
                if euclidean(padded[i] , padded[i+1])>1000:
                    lst.append(padded[i])
    
        
        #cos = euclidean(padded[-3],padded[-1])
        
        if len(lst)>1: 
            res = np.concatenate(lst)
            res = [tok for tok in res if tok!=0]
        elif len(lst)==0:
            res = [tok for tok in padded[0] if tok!=0]
        else:
            res = [tok for tok in lst[0] if tok!=0]
    else:
        res = tokenizer.encode(string, add_special_tokens=True)
        #tokenized_sents[0]
    
    len(res)-2/2
    
    
    
    return res
    
    
    #return padded,
    #return padded, cos

In [79]:
euclidean([0,1,1,2],[0,1,2,1])

1.4142135623730951

In [80]:
df['text'].sample().apply(lambda x: chunker(x))

151120    [101, 2694, 2739, 2121, 2003, 1037, 9927, 2055...
Name: text, dtype: object

def chunker(string):
    

    sents_ = sents(string)
    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_] #####

    usents=[]
    
    for sent in tokenized_sents:
        if sent not in usents:
            usents.append(sent)
    
    if len(string)==5000:
        
        clean_string = sum(usents, [])[:-1]+[102]
    else:
        clean_string = sum(usents, [])
    
 
    clean_len = len(clean_string)
    
    
    words_set = list(set([i for i in clean_string if i not in [101, 102, 100]]))
    ws_len = len(words_set)

    
    counter = 1
    
    for word in words_set:
        word_aq = clean_string.count(word)
        if word_aq > 2:
            counter +=1
    
    if counter/ws_len>0.5:
        clean_string = list(dict.fromkeys(clean_string))
        
    return clean_string

In [82]:
#X_resample['text'].iloc[0:2]

In [ ]:
#X_resample['text'].apply(lambda x: tokenizer.decode(dups(x)))

In [83]:
df.iloc[98437]['text']

"And there's no real difference between a mini-album and an EP, no matter how Korea looks at it."

In [ ]:
#tokenizer.decode(chunker(df.loc[157841]['text']))

In [ ]:
#df.loc[157840:157841]['text'].values.astype('U')[1]
#tokenizer.decode(tokenizer.encode(clean_text(df.loc[157840:157841]['text'].values.astype('U')[1])))

In [ ]:
#tokenizer.decode(tokenizer.encode(clean_text(df.loc[157841]['text'])))

In [ ]:
#tokenizer.decode(chunker(df['text'][4700]))

In [87]:
###tokenizer.decode(chunker(df['text'][83666]))

In [86]:
#X_resample['text'].shape[0]/4

In [185]:
dembs = BertTokenizer(
    text=X_train,
    batch_size=64).get()

  0%|          | 0/8 [00:00<?, ?it/s]

In [186]:
logy(dembs, y_train)

array([0.61236559, 0.85505207, 0.79654255, 0.66090426, 0.76112412])

In [189]:
X_resample.shape

(88,)

In [190]:
dembs = BertTokenizer(
    text=X_resample,
    batch_size=44).get()

  0%|          | 0/2 [00:00<?, ?it/s]

In [192]:
y_resample

68104     1
151100    1
108253    0
33524     1
100864    1
         ..
74751     1
40182     0
147241    1
15603     1
6764      0
Name: toxic, Length: 88, dtype: int64

In [194]:
logy(dembs, y_resample)

array([0.71428571, 0.94427245, 0.94427245, 0.94035088, 0.81318681])

In [ ]:
1/0

In [ ]:
#np.array([0.86706731, 0.89236626, 0.86075392, 0.89853502, 0.87655137])

In [ ]:
#tokenizer.decode(chunker(df[df['text'].apply(lambda x: len(x))>2500]['text'].sample(2).iloc[0]))

In [ ]:
df[df['text'].str.contains("you can suck my lick you", flags = re.IGNORECASE)]

In [ ]:
euclidean(aa,bb)

In [ ]:
np.array([0.85350318, 0.8625    , 0.87179487, 0.82352941, 0.84146341]).mean()

In [ ]:
X_test.head(), y_test.head()

In [ ]:
y_test.shape

In [ ]:
test_sample = pd.DataFrame(X_test).join(pd.DataFrame(y_test)).sample(n=dembs.shape[0], random_state=RANDOM_STATE)

In [ ]:
y_test = 

In [ ]:
test_sample.head()

In [ ]:
#array([0.88311688, 0.88303797, 0.76315789, 0.82868042, 0.82820379])

In [ ]:
test_sample['text'].shape[0]

In [ ]:
#dembs_test = BertTokenizer(text=test_sample['text'], batch_size=191).get()
dembs_test = BertTokenizer(text=X_test, batch_size=64).get()

In [ ]:
y_train.value_counts(normalize=2)

In [ ]:
def f1_eval(y_true, y_pred,):
    #y_true = dtrain.get_label()
    #err = 1 - f1_score(grid_pred, y_test, average=None)[1]
    err = f1_score(y_pred, y_true, average=None)[1]
    return  err

In [ ]:
def f1_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = f1_score(y_true, y_pred, average=None)[1]

    return 'err', err

In [ ]:
from sklearn.metrics import make_scorer
my_func = make_scorer(f1_eval, greater_is_better=True)

In [ ]:
param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        'models__C': [0.1, 1]#,
        #'models__penalty': ['l1', 'l2']
          
    },

#    {
#        'models': [SVC(random_state=RANDOM_STATE, class_weight='balanced')],
#        'models__degree': range(3,8),
#        'models__kernel': ['poly','rbf','sigmoid']
#    },
    {
        'models': [XGBClassifier(random_state=RANDOM_STATE, maximize=True)],
        'models__n_estimators':[500],
        'models__learning_rate':[0.1, 0.07, 0.05],
        'models__eval_metric': [f1_xgb]
    }
    #{
    #    'models': [LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
    #    'models__n_estimators': [1000],
    #    'models__learning_rate': [0.1, 0.01, 0.03, 0.05, 0.07]
    #}
]

param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE)],#, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        'models__C': [0.1,2,3]#,
        #'models__penalty': ['l1', 'l2']
          
    }
]

In [ ]:
pipe = Pipeline([('models',[LogisticRegression(random_state=RANDOM_STATE,class_weight='balanced')])])

In [ ]:
grid_search = GridSearchCV(
    pipe, 
    param_grid, 
    cv=5,
    scoring='roc_auc',
    #scoring=my_func,
    n_jobs=-1
)
#grid_search.fit(X_resample[:X_test.shape[0]], y_resample['toxic'][:X_test.shape[0]])

#grid_search.fit(embs, y_resample['toxic'].iloc[:test_sample.shape[0]])
#grid_search.fit(embs, y_resample)
#grid_search.fit(embs, y_train)
grid_search.fit(dembs, y_train)

In [ ]:
pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'mean_test_score','params']].sort_values('rank_test_score')#.loc[3]['params']#.head(16)

In [ ]:
#grid_search.best_estimator_

In [ ]:
grid_pred = grid_search.best_estimator_.predict(dembs_test)#[:X_train.shape[0]])

In [ ]:
y_test.shape

In [ ]:
dembs_test.shape

In [ ]:
grid_pred.shape

In [ ]:
#0.6818181818181818

In [ ]:
#y_test[test_sample.index]

In [ ]:
#0.5166340508806262
#0.670076726342711

In [ ]:
f1_score(grid_pred, y_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(grid_pred, y_test))

In [ ]:
res =pd.DataFrame({'a':grid_pred, 'b':test_sample['toxic']})

In [ ]:
X_train.apply(lambda x: len(x)).hist()

In [ ]:
X_train.shape

In [ ]:
X_resample.shape

In [ ]:
X_train[X_train.apply(lambda x: len(x))>2000]

In [ ]:
res.shape

In [ ]:
df['text'].

In [ ]:
df[(df.index.isin(res[res['a']!=res['b']].index))&(df['toxic']==1)]['text'].apply(lambda x: tokenizer.decode(chunker(x)))#.sample()



#['b'].value_counts()

In [ ]:
tokenizer.decode(chunker(df.loc[128933]['text']))

In [ ]:
df.loc[128933]['text']

## Обучение

In [ ]:
#from phik.report import plot_correlation_matrix
#from phik import phik_matrix

In [ ]:
#df.info()

phik_overview = phik_matrix(df[['toxic','word_count']],verbose=False)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Blues',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(18, 14)
) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_lemm'], 
    df['toxic'], 
    random_state=RANDOM_STATE,
    stratify=df['toxic']
)

In [ ]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train)) 


In [ ]:
y_resample.value_counts()

In [ ]:
pd.DataFrame(y_test).value_counts()

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны